In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# POC

In [4]:
sentence1='This is sentence one'
sentence2='This is sentence two'
sentence3='Not a matching statement'

df=pd.Series([sentence1,sentence2,sentence3])

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

data=cv.fit_transform(df)

sim=cosine_similarity(data)

#print(sim)
result=[]
for row in df.index:
    print(row,'\t',df[row],'\t',sim[row],'\t',sum(sim[row]))
    result.append( [df[row],sum(sim[row])] )

sorted(result,reverse=True)[0:2]

# So these two are similar

0 	 This is sentence one 	 [1.   0.75 0.  ] 	 1.75
1 	 This is sentence two 	 [0.75 1.   0.  ] 	 1.75
2 	 Not a matching statement 	 [0. 0. 1.] 	 1.0000000000000002


[['This is sentence two', 1.75], ['This is sentence one', 1.75]]

In [5]:
df=pd.read_csv('Worklist_abstract.csv')
print(df.shape)
print(df.drop_duplicates().shape)
# NO duplicates

(4500, 6)
(4500, 6)


In [6]:
df.head()

,Ticket #,Severity,Next Check Time,Ticket State,Last Modified Date,Problem Abstract
0,116545208,3 - Minor,09/12/2009 01:41:27 AM,Deferred - Other,09/25/2009 11:35:04 PM,Advanced TV has no network connectivity. Need...
1,116821366,2 - Major,09/24/2009 03:52:38 AM,Deferred - Other,03/24/2010 07:38:51 AM,PDC UVERSE ORDER ISSUES
2,116868930,3 - Minor,09/26/2009 05:09:15 AM,Deferred - Other,10/12/2009 11:07:19 PM,APPLICATION ISSUES
3,118026344,3 - Minor,11/13/2009 03:13:39 AM,Deferred - Other,11/18/2009 04:31:20 AM,Register just re-imaged by local IT and it doe...
4,118723231,5 - Normal,12/09/2009 03:07:10 AM,Deferred - Other,08/28/2010 01:58:54 AM,Z - OTHER SOE ISSUES


In [7]:
df.info()
# some ticket description are null, no use, can be dropped

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Ticket  #           4500 non-null   int64 
 1   Severity            4500 non-null   object
 2   Next Check Time     4500 non-null   object
 3   Ticket State        4500 non-null   object
 4   Last Modified Date  4500 non-null   object
 5   Problem Abstract    4469 non-null   object
dtypes: int64(1), object(5)
memory usage: 211.1+ KB


In [8]:
len(df[df['Ticket State']=='Active'])
#401 records are active that can be used for testing

401

# Data cleaning

In [9]:
# Last Modified Date,Next Check Time : Irrelevant info
df.drop(['Last Modified Date','Next Check Time'],axis=1,inplace=True)

In [10]:
df.head()
#need to simplify column name
# Convert to numerical =>severitity col
# Ticket state - Categorical. just take care of deferred status

,Ticket #,Severity,Ticket State,Problem Abstract
0,116545208,3 - Minor,Deferred - Other,Advanced TV has no network connectivity. Need...
1,116821366,2 - Major,Deferred - Other,PDC UVERSE ORDER ISSUES
2,116868930,3 - Minor,Deferred - Other,APPLICATION ISSUES
3,118026344,3 - Minor,Deferred - Other,Register just re-imaged by local IT and it doe...
4,118723231,5 - Normal,Deferred - Other,Z - OTHER SOE ISSUES


In [11]:
df.columns=['ticket', 'severity', 'status', 'description']
df.head(2)

,ticket,severity,status,description
0,116545208,3 - Minor,Deferred - Other,Advanced TV has no network connectivity. Need...
1,116821366,2 - Major,Deferred - Other,PDC UVERSE ORDER ISSUES


In [12]:
df=df[df['description'].notnull()]
print(df.shape)

(4469, 4)


## Second csv file for tickets

In [13]:
tracker=pd.read_csv('Tracker_simple.csv',encoding='ISO-8859-1')  #some char not decoded by UTF-8 format
tracker.head()

,Application,Status,Ticket,Description
0,CAPM,Assigned To PS-T1,258593331,The ptauth response files are taking to long t...
1,CAPM,Assigned To PS-T1,240050653,profileMaint.LOG.CAPM.B20180101 failure
2,CAPM,Assigned To PS-T1,240108084,Job failure : achpam.c010218220309
3,CAPM,Assigned To PS-T1,240196247,chgbkFD.f010518035553 failure
4,CAPM,Assigned To PS-T1,240196255,achpam.c010418220326 file failure


In [14]:
tracker.columns=['application','status','ticket','description']

In [15]:
tracker.head()

,application,status,ticket,description
0,CAPM,Assigned To PS-T1,258593331,The ptauth response files are taking to long t...
1,CAPM,Assigned To PS-T1,240050653,profileMaint.LOG.CAPM.B20180101 failure
2,CAPM,Assigned To PS-T1,240108084,Job failure : achpam.c010218220309
3,CAPM,Assigned To PS-T1,240196247,chgbkFD.f010518035553 failure
4,CAPM,Assigned To PS-T1,240196255,achpam.c010418220326 file failure


In [16]:
tracker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   application  1205 non-null   object
 1   status       1205 non-null   object
 2   ticket       1205 non-null   int64 
 3   description  1205 non-null   object
dtypes: int64(1), object(3)
memory usage: 37.8+ KB


In [17]:
print('Original shape :          ',tracker.shape)
print('After dropping duplicates:',tracker.drop_duplicates(keep='first').shape)
print(tracker[tracker.duplicated('ticket')]['ticket'].count())
# 1205-1187 = 18 seems duplicates present if all column considered
# 37 seems duplicates present if ONLY 'ticket' considered

Original shape :           (1205, 4)
After dropping duplicates: (1187, 4)
37


In [18]:
# need to check if ticket duplicates in both dataset
len(tracker[tracker.duplicated('ticket')].sort_values(by='ticket'))

#here only first occurence is visible => total : 37

37

In [19]:
tracker.iloc[1200]

application                                                CAPM 
status                                         Assigned To PS-T1
ticket                                                 271355923
description    Missing credit response ref error on failed tr...
Name: 1200, dtype: object

In [20]:
tracker[tracker['ticket']==271355923]

#244845137 all rows same
#271355923 only ticket same, desc diff

,application,status,ticket,description
266,CAPM,Assigned To PS-T1,271355923,OTHER
1200,CAPM,Assigned To PS-T1,271355923,Missing credit response ref error on failed tr...


In [21]:
cnt=0
for row in tracker.values:
    if row[3]=='OTHER':
        print(row[2])
        cnt+=1
print(cnt)
# description with just "OTHER" doesn't help us. So removing

271355923
271218150
270965451
3


In [22]:
print(tracker.shape)
tracker=tracker[tracker['description']!='OTHER']
print(tracker.shape)

(1205, 4)
(1202, 4)


In [23]:
#Before this step, drop tickets with "OTHER" values!!! Else we drop ticket with desc over ticket with just Other
tracker=tracker.drop_duplicates('ticket')
tracker.shape

#dropped 36 same tickets

(1166, 4)

In [24]:
# Same for first data =>df

cnt=0
for row in df.values:
    if row[3]=='OTHER':
        #print(row[3])
        cnt+=1
print(cnt)
print(df.shape)
# 208 records with description 'OTHER'

208
(4469, 4)


In [25]:
df=df[df['description']!='OTHER']

df=df.drop_duplicates('ticket')
df.shape


# 4469-4261=208 records with OTHER dropped

(4261, 4)

In [26]:
df.to_csv('data/AOTS.csv',index=False)
tracker.to_csv('data/tracker.csv',index=False)

# Backup of clean data

In [27]:
df=pd.read_csv('data/AOTS.csv')
tracker=pd.read_csv('data/tracker.csv')

In [28]:
print(df.shape)
print(tracker.shape)

(4261, 4)
(1166, 4)


In [29]:
df.head(2)

,ticket,severity,status,description
0,116545208,3 - Minor,Deferred - Other,Advanced TV has no network connectivity. Need...
1,116821366,2 - Major,Deferred - Other,PDC UVERSE ORDER ISSUES


In [30]:
tracker.head(2)

,application,status,ticket,description
0,CAPM,Assigned To PS-T1,258593331,The ptauth response files are taking to long t...
1,CAPM,Assigned To PS-T1,240050653,profileMaint.LOG.CAPM.B20180101 failure


Since we don't have application in one dataset & severity in other
lets drop them & merge both

In [31]:
df.drop('severity',inplace=True,axis=1)
tracker.drop('application',inplace=True,axis=1)

In [32]:
df.head(2)

,ticket,status,description
0,116545208,Deferred - Other,Advanced TV has no network connectivity. Need...
1,116821366,Deferred - Other,PDC UVERSE ORDER ISSUES


In [33]:
tracker=tracker[['ticket','status','description']]
tracker.head(2)

,ticket,status,description
0,258593331,Assigned To PS-T1,The ptauth response files are taking to long t...
1,240050653,Assigned To PS-T1,profileMaint.LOG.CAPM.B20180101 failure


In [34]:
# Tried merging these two df on ticket no. , No common values ie no duplicates

In [35]:
comb=pd.concat([df,tracker],ignore_index=True)

In [36]:
comb.head()

,ticket,status,description
0,116545208,Deferred - Other,Advanced TV has no network connectivity. Need...
1,116821366,Deferred - Other,PDC UVERSE ORDER ISSUES
2,116868930,Deferred - Other,APPLICATION ISSUES
3,118026344,Deferred - Other,Register just re-imaged by local IT and it doe...
4,118723231,Deferred - Other,Z - OTHER SOE ISSUES


In [37]:
print(comb.shape)
print(comb.drop_duplicates().shape)

(5427, 3)
(5427, 3)


In [38]:
comb['status'].value_counts()

Queued                            2952
Assigned To PS-T1                 1151
Ready to Close                     658
Active                             385
Cleared                             79
Closed                              67
Deferred - Other                    38
Cancel                              36
Deferred - Cust Information         31
Active - DVCI                       13
Deferred - Cust Vendor               9
Deferred - Delayed Maintenance       4
Active - SME                         2
Deferred - Pend Recurrence           2
Name: status, dtype: int64

In [39]:
def status_mapping(status):
    if status in ['Active','Active - DVCI','Active - SME']:
        return 'Active'
    else:
        return 'Resolved'

In [40]:
comb['status']=comb['status'].apply(status_mapping)

In [41]:
comb['status'].value_counts()

Resolved    5027
Active       400
Name: status, dtype: int64

In [42]:
comb.to_csv('data/comb.csv',index=False)
comb.sample(2)

,ticket,status,description
3997,213925769,Active,CLOSE KEY TRACKING ONLY
2944,210027742,Resolved,MY ATT SERVICES


In [43]:
train=comb[comb['status']=='Resolved']

In [44]:
# This much we have data for old tickets that are sovled : reference
train.shape

(5027, 3)

In [45]:
train['description']=train['description'].apply(lambda x : re.sub('[^a-zA-Z0-9$-]', ' ', x) )
train['description']=train['description'].apply(lambda x : x.lower())

C:\Users\akashmas\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\akashmas\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
train['description']

0       advanced tv has no network connectivity   need...
1                                 pdc uverse order issues
2                                      application issues
3       register just re-imaged by local it and it doe...
4                                    z - other soe issues
                              ...                        
5422    missing credit response ref error on failed tr...
5423    error for reports ran from webcapm in spring mode
5424      ob westunion genwuaccts accountinfo job failure
5425    distribution report the report is not showing ...
5426    ar plans for 3 subs failed with the error - pa...
Name: description, Length: 5027, dtype: object

In [47]:
cv = CountVectorizer(stop_words='english') #stop_words='english'
count_matrix = cv.fit_transform(train['description'])

In [48]:
print(len(cv.vocabulary_),'vocabulary words')
print(count_matrix.shape,' shape of matrix')

4366 vocabulary words
(5027, 4366)  shape of matrix


In [49]:
sim = cosine_similarity(count_matrix)

In [50]:
#ans[0:4]
#train['ticket'][9].values

In [51]:
# Checking by taking an example

m='APPLICATION ISSUES'
m=m.lower().strip()

# getting the index of the ticket in the dataframe
i = train.loc[train['description']==m].index[0]

lst = list(enumerate(sim[i]))

# sorting this list in decreasing order based on the similarity score
lst = sorted(lst, key = lambda x:x[1] ,reverse=True)

# taking top 1- Tickets
# not taking the first index since it is the same Tcket
lst = lst[1:3]

#print('lst[0]:',lst[0])
#print('lst[0][0]:',lst[0][0])

# making an empty list that will containg all the similar tickets nos
ans = []
for i in range(len(lst)):
    a = lst[i][0]
    ans.append(train.iloc[a].values)   #,'status','description'


In [52]:
train['ticket'][37]
train[train['ticket']==243087989]

# Previously, while comining two dfs, i was not ignoring individual indexes, hence two ticket with same index existed.
# hence added ignore_index=True above

,ticket,status,description
4298,243087989,Resolved,settlegcs batch cc fcp 1 20180320130036 failed


In [53]:
#print out the list
for rec in ans:
    print(rec)

[149945647 'Resolved' 'application issues - general']
[176248643 'Resolved' 'all issues']
